# Test Choices in K and Interpolation

In [1]:
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.time import Time
from astropy.constants import c
from scipy import interpolate
from scipy.optimize import minimize, least_squares, curve_fit
from mpfit import mpfit

from tqdm import tqdm
import seaborn as sns

from waveCal import *

In [ ]:
# Gather files

# LFC
lfc_files = glob('/mnt/home/lzhao/ceph/lfc5*/LFC_*.fits')
ckpt_files = glob('/mnt/home/lzhao/ceph/ckpt5*/LFC_19*.npy')
lfc_files, lfc_times = sortFiles(lfc_files, get_mjd=True)
ckpt_files = sortFiles(ckpt_files)
num_lfc_files = len(lfc_files)

hdus = fits.open(lfc_files[0])
t_spec = hdus[1].data['spectrum'].copy()
t_errs = hdus[1].data['uncertainty'].copy()
t_mask = hdus[1].data['pixel_mask'].copy()
hdus.close()
nord, npix = t_spec.shape

lfc_orders = range(45,76)

In [ ]:
# ThAr
thar_files = glob('/mnt/home/lzhao/ceph/thar5*/ThAr_*.fits')
thid_files  = glob('/mnt/home/lzhao/ceph/thid5*/ThAr_*.thid')
thar_files, thar_times = sortFiles(thar_files, get_mjd=True)
thid_files = sortFiles(thid_files)
num_thar_files = len(thar_files)

In [ ]:
patch_dict = pcaPatch(ckpt_files, file_times=lfc_times,
    K=2, running_window=9, num_iters=45, return_iters=True,
    line_cutoff=0.5, file_cutoff=0.5, fast_pca=False, plot=False, verbose=True)

## Interpolate in Time

In [ ]:
print(patch_dict.keys())

In [ ]:
thid_files = []
thid_wfits = []
thid_diffs = []
for nfile in tqdm(range(len(thid_files))):
    file_name = thid_files[nfile]
    try:
        x,m,w = readThid(file_name)
    except ValueError:
        continue
    w_fit = interp_coefs_and_predict(thar_times[nfile],patch_dict,
                                     interp_deg=1, new_x=x, new_m=m)
    
    thid_files.append(os.path.basename(file_name))
    thid_wfits.append(w_fit)
    thid_diffs.append(w-w_fit)